# Computación cuántica práctica
Talleres que molan. Facultad de Informática, Universidad Complutense de Madrid. 12/NOV/2020.
(c) Alejandro Pozas Kerstjens

physics [a] alexpozas.com

-------
## Introducción
El principio que subyace no sólo la computación cuántica, sino todas las [tecnologías cuánticas de la información](https://qt.eu/), es que **la información es un concepto físico**. Para ser procesada y transmitida, la información tiene que estar codificada y almacenada en sistemas físicos (en cartones perforados, en dominios magnéticos, en pulsos de luz...). Debido a esto, el qué podemos hacer con la información viene determinado por las leyes de la física que rigen el comportamiento de los sistemas en los que ésta está codificada. 

En el intento de codificar información en sistemas físicos cada vez más pequeños, comenzaron a surgir preguntas como ¿Seremos alguna vez capaces de codificar un bit en un único átomo? Dado que el comportamiento de átomos individuales viene dado por la física cuántica, ¿cómo afectarán esas nuevas leyes de la naturaleza al procesamiento de la información? ¿Existirán tareas que no podamos hacer usando sistemas que se rijan por las leyes de la física clásica? ¿Dará la mecánica cuántica ventajas o inconvenientes para tareas de comunicación, computación...?
La consideración de estas preguntas dio lugar al campo de la [teoría cuántica de la información](https://en.wikipedia.org/wiki/Quantum_information). A pesar de que el procesamiento de la información codificada en sistemas cuánticos sufre de varios inconvenientes que no aparecen cuando tratamos con sistemas clásicos (por ejemplo, [la información codificada en un sistema cuántico no se puede copiar](https://es.wikipedia.org/wiki/Teorema_de_no_clonaci%C3%B3n)), muchos han sido transformados en ventajas que permiten, por ejemplo, [protocolos criptográficos completamente seguros](https://core.ac.uk/download/pdf/82447194.pdf) (en comparación con protocolos computacionamlmente seguros como RSA).

### En este taller
En este taller vamos a hacer una introducción a la computación cuántica inspirada en la computación clásica. Veremos, poco a poco, qué características nos proporciona la física cuántica y cómo lidiar con ellas para nuestro provecho. El objetivo es ver un ejemplo práctico de cómo el codificar y procesar información en sistemas cuánticos puede dar una ventaja computacional frente al procesamiento en sistemas clásicos. Veremos un algoritmo cuántico en el que esta ventaja es muy evidente: el [algoritmo de Deutsch-Josza](https://es.wikipedia.org/wiki/Algoritmo_de_Deutsch-Jozsa) para ver si una función booleana desconocida es balanceada o constante. Este problema no tiene muchas aplicaciones en la vida real, pero muestra muy claramente el tipo de ventajas que puede ofrecer la computación cuántica, y nos ayudará a introducir las herramientas de computación cuántica que se están utilizando a día de hoy tanto en investigación como en empresas.

Utilizaremos la librería [Qiskit](http://www.qiskit.org), de código abierto y desarrollada por IBM para tener una interfaz con sus ordenadores cuánticos. Veremos cómo definir circuitos, aplicar puertas lógicas, hacer mediciones, y ejecutar los programas en simuladores y en ordenadores cuánticos reales.

Para comenzar, como en cualquier programa de Python, debemos importar las librerías necesarias. Dado que estamos utilizando Colab, antes de eso, debemos instalarlas en el servidor.

In [ ]:
!pip install qiskit

In [ ]:
import matplotlib.pyplot             # Para dibujar circuitos
from qiskit import Aer, IBMQ         # Simulador y acceso a los chips cuánticos
from qiskit import QuantumCircuit, ClassicalRegister    # Para crear circuitos
from qiskit import execute           # Para ejecutar circuitos

## Building blocks de un programa cuántico
### Circuitos y medidas
Tal y como hemos dicho, el sistema cuántico más pequeño en el que se puede codificar información es el bit cuántico o qubit. En Qiskit, los qubits son procesados en circuitos, que son los objetos básicos de la librería. Consecuentemente, para introducir información en un registro de qubits y procesar la misma, lo primero que debemos hacer es definir un `QuantumCircuit` y declarar cuántos qubits queremos que contenga. De momento, vamos a contentarnos con un solo qubit:

In [ ]:
n_qubits = 1
qubit    = QuantumCircuit(n_qubits)

Pictóricamente, cada qubit representará una línea sólida en el circuito, a las cuales se irán aplicando puertas lógicas, que normalmente aparecen en forma de cajas. Para ver la representación de un circuito particular, podemos llamar a su atributo `draw`.

In [ ]:
qubit.draw(output="mpl")

Antes de entrar a las puertas lógicas, hay que mencionar una característica particular que ocurre al procesar información en sistemas cuánticos. Al final de nuestra computación, queremos tener un resultado, idealmente en forma de una cadena de bits (clásica). Eso significa que tenemos que **realizar mediciones** en los sistemas cuánticos. Como veremos más adelante, esto no es tan trivial como pueda parecer a simple vista ;).

Las mediciones sobre qubits se almacenan en un registro de bits clásicos, que se especifica al generar el``QuantumCircuit``. Después, las mediciones se realizan llamando a la función ``measure``, en la cual se debe especificar el qubit sobre el que se realiza la medida, y el bit en el cual se debe almacenar el resultado de dicha medida.

In [ ]:
n_bits = 1
qubit_circuit = QuantumCircuit(n_qubits, n_bits)
# Medimos el qubit número 0 y almacenamos el resultado en el bit número 0
qubit_circuit.measure(qubit=0, cbit=0)
qubit_circuit.draw(output="mpl")

Una última cosa que debemos ver antes de empezar a aplicar puertas lógicas es cómo ejecutar un circuito. Esto se hace mediante la función `execute` que, a su vez, necesita de la especificación de un *backend*. Es decir, necesitamos especificar la plataforma en la cual se ejecutará el circuito. Existen tres *backends* de gran utilidad:

- El ```statevector_simulator```. Es un simulador de estados cuánticos que utiliza álgebra lineal para computar el estado resultado de un algoritmo, aplicando la matriz unitaria correspondiente al circuito para dar lugar al resultado, que en este caso es también un estado cuántico.

- El ```qasm_simulator```. Es un simulador clásico de un ordenador cuántico real. El funcionamiento es similar al ```statevector_simulator```, pero el output no es el estado cuántico final, sino medidas realizadas sobre este estado. Es decir, el output son cadenas de bits que representan los resultados de las medidas que realizamos a lo largo del circuito.

- Chips cuánticos reales. Cada uno tiene su propio nombre, y veremos más adelante cómo llamar a cada uno de ellos.

Los simuladores se encuentran en el módulo ```Aer``` de ```qiskit```, mientras que para acceder a los chips reales necesitamos el módulo ```IBMQ```. Comencemos con los simuladores.

In [ ]:
statevector_simulator = Aer.get_backend("statevector_simulator")
chip_simulator        = Aer.get_backend("qasm_simulator")

In [ ]:
qubit_statevector = execute(qubit, statevector_simulator)
print(qubit_statevector.result().get_statevector())

El estado cuántico de nuestro qubit inicial es un vector de dos componentes. Estas son las *amplitudes de probabilidad* de estar en el estado correspondiente al $0$ lógico y de estar en el estado correspondiente al $1$ lógico. En general, el estado de un qubit puede describirse con un vector $|\psi\rangle=(c_0, c_1)=c_0|0\rangle + c_1|1\rangle$, donde $c_0$ y $c_1$ son números complejos que satisfacen $|c_0|^2+|c_1|^2=1$. En el caso de arriba, tenemos que $c_0=1$ y $c_1=0$, de modo que el qubit está en el estado $|\psi_{\text{inic}}\rangle = |0\rangle$. De hecho, al iniciar un circuito, *todos los qubits comienzan preparados en el estado $|0\rangle$*. Sin embargo, el hecho de que podamos tener *superposiciones* de los estados $|0\rangle$ y $|1\rangle$ es una de las características que dota a la computación cuántica de gran poder, como veremos más adelante.

Veamos ahora el resultado de la ejecución segundo circuito, en el cual habíamos introducido una medida. Este es el tipo de circuitos que se envían a ejecutar a ordenadores cuánticos, siendo el resultado una cadena de bits clásicos.

In [ ]:
qubit_circuit_sim = execute(qubit_circuit, chip_simulator)
print(qubit_circuit_sim.result().get_counts())

El resultado indica que se ha ejecutado el circuito un total de $1024$ veces, y en todas ellas el valor medido del qubit ha sido el valor $0$. El qubit estaba inicializado en el estado $|0\rangle$, así que tiene bastante sentido que al medir su estado, obtengamos el valor $0$ cada vez que lo medimos. Pero, si hacemos tanto énfasis en esto, posiblemente es porque haya algo interesante detrás ;)

### Aplicando puertas lógicas
Ya hemos visto cómo crear circuitos cuánticos, medir qubits y almacenar los resultados obtenidos, y ejecutar dichos circuitos en simuladores. A continuación veremos cómo aplicar transformaciones a los qubits, para implementar operaciones y algoritmos.

En Qiskit, las puertas lógicas se aplican mediante llamadas a funciones atributo del ``QuantumCircuit``. Empecemos por algo sencillo. La puerta lógica NOT, que cambia el estado $0$ por el $1$ y viceversa, se denomina en computación cuántica la *puerta $X$*. Veamos un simple circuito en el que se ve la aplicación de dicha puerta:

In [ ]:
x_circuit = QuantumCircuit(1, 2)    # Definimos el circuito con 1 qubit y 2 bits
x_circuit.measure(0, 0)             # Medimos el estado del qubit
x_circuit.x(q=0)                    # Aplicamos la puerta X en el qubit 0
x_circuit.measure(0, 1)             # Medimos el estado del qubit de nuevo
x_circuit.draw(output="mpl")

In [ ]:
execute(x_circuit, chip_simulator).result().get_counts()

Ahora vemos que, todas las $1024$ veces, el resultado de la primera medida es $0$, y el resultado de la segunda medida, que hemos almacenado en el segundo bit (sí, los registros están numerados de derecha a izquierda) es siempre $1$.

Vamos ahora con la primera puerta realmente cuántica, la *puerta de Hadamard*. Esta puerta lógica crea estados en superposición, puesto que transforma el estado $|0\rangle$ en el estado $\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$, y el estado $|1\rangle$ en el estado $\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$. Veámoslo explícitamente:

In [ ]:
# Aplicación de la puerta de Hadamard al estado |0>
hadamard_from_zero = QuantumCircuit(1)
hadamard_from_zero.h(0)
print(execute(hadamard_from_zero, statevector_simulator).result().get_statevector())

# Aplicación de la puerta de Hadamard al estado |1>
hadamard_from_one = QuantumCircuit(1)
hadamard_from_one.x(0)
hadamard_from_one.h(0)
print(execute(hadamard_from_one, statevector_simulator).result().get_statevector())

¿Qué pasará cuando hagamos una medición en estos estados? Veámoslo

In [ ]:
hadamard_from_zero.add_register(ClassicalRegister(1))
hadamard_from_zero.measure(0, 0)
print(execute(hadamard_from_zero, chip_simulator, shots=10000).result().get_counts())

hadamard_from_one.add_register(ClassicalRegister(1))
hadamard_from_one.measure(0, 0)
print(execute(hadamard_from_one, chip_simulator, shots=10000).result().get_counts())

Parece ser que, en ambos casos, más o menos la mitad de las veces el resultado de la medida es $0$, y la otra mitad de las veces es $1$. Esto es debido a que **la física cuántica describe una naturaleza probabilista**. Cuando un qubit (o cualquier sistema cuántico) se encuentra en un estado de superposición y se realiza una medición sobre él, el resultado de dicha medición es una de las posibles opciones de la superposición, con una probabilidad asociada a la correspondiente amplitud de probabilidad. Esto es, si realizamos una medida en un qubit preparado en el estado $|\psi\rangle=c_0|0\rangle + c_1|1\rangle$, el resultado de dicha medida será $0$ con una probabilidad $|c_0|^2$, y $1$ con una probabilidad $|c_1|^2$. En el caso de las superposiciones tras la puerta de Hadamard, estas probabilidades son $\left|\pm\frac{1}{\sqrt{2}}\right|^2=\frac{1}{2}$.

Antes de ver puertas lógicas que involucren dos qubits, vamos a ver una última característica de la física cuántica

In [ ]:
def which_qubit_state(statevec):
  basis_states = ["|0>", "|1>"]
  state = ""
  for amplitude, basis_state in zip(statevec, basis_states):
    if abs(amplitude - 1) < 1e-8:
      state += "+" + basis_state
    elif abs(amplitude) < 1e-8:
      continue
    else:
      if str(amplitude)[0] == "-":
        state += str(round(amplitude, 4)) + basis_state
      else:
        state += "+" + str(round(amplitude, 4)) + basis_state
  return state[1:] if state[0] != "-" else state

for _ in range(10):
  program = execute(hadamard_from_zero, statevector_simulator, memory=True)
  result  = program.result()
  measurement_result = result.get_memory()
  statevector_result = result.get_statevector()
  print("El resultado de la medida fue " + measurement_result[0]
        + " y el estado tras la medida es " + which_qubit_state(statevector_result))

Cuando realizamos una medición, el estado cambia al correspondiente al resultado de la medida. Este fenómeno se llama [colapso de la función de onda](https://es.wikipedia.org/wiki/Colapso_de_la_funci%C3%B3n_de_onda), y es un proceso puramente cuántico sin análogo en el mundo clásico. En el ámbito de la computación cuántica, lo que nos importa es saber que **realizar medidas destruye las superposiciones**, de modo que deberemos realizar mediciones únicamente cuando dichas superposiciones no existan, o en lugares donde tengamos muy controlado qué está ocurriendo, y deseemos que un colapso ocurra.

Para terminar con la introducción a puertas lógicas cuánticas, vamos a ver que se pueden realizar puertas que involucran a varios qubits. Un ejemplo muy importante es la puerta CNOT. Ésta es una puerta lógica aplicada a dos qubits, que aplica la puerta $X$ al segundo qubit (el qubit objetivo), solamente si el primer qubit (llamado qubit de control) está en el estado $|1\rangle$. Si el qubit de control está en el estado $|0\rangle$, entonces el qubit objetivo se mantiene intacto. Es decir, la puerta CNOT realiza las siguientes operaciones:

$$
\begin{align}
CNOT(|0\rangle|0\rangle)&=|0\rangle|0\rangle\\
CNOT(|0\rangle|1\rangle)&=|0\rangle|1\rangle\\
CNOT(|1\rangle|0\rangle)&=|1\rangle|1\rangle\\
CNOT(|1\rangle|1\rangle)&=|1\rangle|0\rangle
\end{align}
$$

O, en una línea, $CNOT(|x\rangle|y\rangle)=|x\rangle|y\oplus x\rangle$.

**Ejercicio:** Utilizando las puertas $X$, Hadamard y CNOT, crea un circuito que genere el estado cuántico $\frac{1}{\sqrt{2}}(|0\rangle|1\rangle-|1\rangle|0\rangle)$, y que después mida el estado de los dos qubits. Ejecuta el programa varias veces. ¿Qué observas en los resultados de las medidas? Eso que acabas de observar es el [entrelazamiento cuántico](https://es.wikipedia.org/wiki/Entrelazamiento_cu%C3%A1ntico).

A modo de conclusión de esta parte, se debe mencionar que no todo son $X$, Hadamard y CNOT. De hecho, el modelo de circuitos cuánticos es un [modelo de computación universal](https://en.wikipedia.org/wiki/Quantum_logic_gate#Universal_quantum_gates), es decir, existen conjuntos de puertas lógicas que permiten escribir cualquier computación como circuitos que únicamente utilizan puertas de uno de estos conjuntos. Las puertas implementables en Qiskit son suficientes para formar un conjunto universal. Todas las puertas que se pueden insertar en un circuito cuántico se pueden ver en la [documentación](https://qiskit.org/documentation/api/qiskit.circuit.QuantumCircuit.html), aunque una descripción más amable de las puertas más comunes se puede encontrar [aquí](https://community.qiskit.org/textbook/ch-gates/quantum-gates.html).

**Ejercicio**: Crea los estados cuánticos $|GHZ\rangle=\frac{1}{\sqrt{2}}\left(|000\rangle+|111\rangle\right)$ y $|W\rangle=\frac{1}{\sqrt{3}}\left(|100\rangle+|010\rangle+|001\rangle\right)$. El estado GHZ es posible de preparar solo con puertas CNOT y Hadamard. Crear el [estado W](https://en.wikipedia.org/wiki/W_state) es menos obvio, así que puedes implementar el circuito en la respuesta a [esta pregunta](https://physics.stackexchange.com/questions/311743/quantum-circuit-for-a-3-qubit-w-rangle-state). Tres apuntes: primero, la puerta que en la respuesta se llama $G(1/3)$ se puede implementar a través de una puerta $U3$ de qiskit (recuerda mirar [la colección de puertas comunes](https://community.qiskit.org/textbook/ch-gates/quantum-gates.html) y la [documentación](https://qiskit.org/documentation/api/qiskit.circuit.QuantumCircuit.html)). Segundo, las puertas controladas se activan *cuando el qubit de control está en el estado $|0\rangle$*, mientras que las puertas normales se activan cuando el qubit de control está en el estado $|1\rangle$. Para cambiar de estado de activación basta con poner una puerta $X$ en cada qubit de control antes y después de la puerta controlada. Tercero, la última puerta, que es una CNOT con dos qubits de control, se llama [puerta de Toffoli](https://en.wikipedia.org/wiki/Toffoli_gate).

## Aplicación "práctica": El algoritmo de Deutsch-Jozsa
Veremos un algoritmo cuántico en el que la ventaja de la computación cuántica es muy evidente, pero que por desgracia resuelve un problema que no tiene una gran aplicabilidad: el [algoritmo de Deutsch-Josza](https://es.wikipedia.org/wiki/Algoritmo_de_Deutsch-Jozsa) para ver si una función booleana desconocida es balanceada o constante. Este problema no tiene muchas aplicaciones en la vida real, pero muestra muy claramente el tipo de ventajas que puede ofrecer la computación cuántica.

El problema que resuelve el algoritmo de Deutsch-Jozsa es el siguiente: imaginemos que nos dan una función sobre cadenas de bits de una longitud determinada, $n$, a un solo bit, $f: \{0,1\}^n\rightarrow\{0,1\}$. Desconocemos la forma específica de dicha función, pero sabemos una cosa: o siempre da el mismo resultado, independientemente del input (es decir, es una función *constante*), o para la mitad de los inputs el correspondiente output es $0$ mientras que para la otra mitad el output es $1$ (es decir, es una función *balanceada*). Por ejemplo, ejemplos de una función de dos bits balanceada y otra constante son los siguientes:

| $b_0$ | $b_1$ | $f_\text{balan}(b_0,b_1)$ | $f_\text{const}(b_0,b_1)$ |
|-------|-------|:------------------------------:|:------------------------------:|
|  $0$  |  $0$  |     $1$                      |     $1$                     |
|  $0$  |  $1$  |     $0$                      |     $1$                     |
|  $1$  |  $0$  |     $1$                      |     $1$                     |
|  $1$  |  $1$  |     $0$                      |     $1$                     |


¿Cómo es posible saber si una $f$ dada es constante o balanceada? Si tenemos un ordenador clásico, sólo hay una opción posible: Ir calculando el valor de $f$ para distintos inputs, e ir viendo qué outputs van saliendo. De hecho, esta estrategia puede funcionar muy rápidamente. Si tenemos suerte, para el primer input que insertemos obtendremos el valor $0$ (por ejemplo), y para el segundo, el valor $1$. Entonces sabemos que $f$ no es constante, y por tanto, debe ser balanceada (recordad que sabemos que la función es o constante o balanceada, y si no es de un tipo, ha de ser del otro).

Pero, de la misma manera, las cosas pueden ir muy mal. Lo peor que puede ocurrir es que, al ir probando inputs diferentes, vamos obteniendo siempre el valor $0$, hasta haber probado la mitad de todos los inputs posibles. A pesar de haber obtenido todos los resultados iguales, aún no podríamos decir nada acerca de la función, porque podría darse el caso de que hubieramos tenido la 'suerte' de haber escogido todos los inputs que dan el valor $0$, y todos los que nos quedaran por probar a continuación darían el valor $1$. Así que, en el peor de los casos, tenemos que probar la mitad de todos los inputs posibles más uno. Para cadenas de bits de longitud $n$, esto es $2^{n-1}+1$. Por ejemplo, si estuviésemos evaluando funciones sobre cadenas de $100$ bits, deberíamos probar $633825300114114700748351602687$ ($633$ octillones) de inputs antes de tener una respuesta concluyente.

### El algoritmo
El algoritmo de Deutsch y Jozsa utiliza la *superposición* para reducir el número de operaciones necesarias para determinar si $f$ es balanceada o constante. Igual que un qubit podía encontrarse en un estado de superposición de sus dos "estados clásicos" posibles (los estados $|0\rangle$ y $|1\rangle$), un sistema de $n$ qubits puede encontrarse en una superposición de hasta $2^n$ estados, y es posible aplicar puertas lógicas a todos estos estados a la vez. De hecho, el algoritmo de Deutsch-Jozsa utiliza la superposición para resolver el problema de la función balanceada utilizando **únicamente una** evaluación de $f$, independientemente de su dominio. El algoritmo tiene la siguiente forma:

![picture](http://dkopczyk.quantee.co.uk/wp-content/uploads/2018/03/circuit_dj-768x159.png)

Vayamos poco a poco viendo qué significan todos estos símbolos. Al comienzo tenemos $n$ qubits inicializados en el estado $|0\rangle\otimes|0\rangle\otimes\dots|0\rangle = |0\rangle^{\otimes n}$, y un qubit adicional también en el estado $|0\rangle$. Es decir, necesitamos utilizar un total de $n+1$ qubits.

Lo primero que hacemos es aplicar una puerta $X$ para poner el último qubit en el estado $|1\rangle$. El estado de los qubits tras la aplicación de esta puerta es 

$$|\psi\rangle=|0\rangle^{\otimes n}|1\rangle$$

A continuación, aplicamos la puerta de Hadamard en cada qubit para crear un enorme estado en superposición. Tras las aplicaciones, acabamos obteniendo el estado

$$|\psi\rangle=\left[\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)\right]^{\otimes n}\otimes\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)=\frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} |x\rangle(|0\rangle - |1\rangle)$$

Como se puede ver, el estado de los qubits ahora contiene una superposición de todas las posibles configuraciones, cada una con el mismo "peso" $2^{-(n+1)/2}$.

Ahora es el momento de implementar la función. Las puertas lógicas son operaciones físicas aplicadas al sistema cuántico y, al estar éste en un estado de superposición, dichas operaciones serán aplicadas a todos los elementos de la superposición. Un método muy común en computación cuántica es utilizar qubits extra para almacenar los resultados de los cálculos. Diseñaremos nuestra función para que, para el estado de entrada $|x\rangle|y\rangle$ genere el estado de salida $|x\rangle|y\oplus f(x)\rangle$. Tras ello, el sistema se encuentra en el estado

$$|\psi\rangle=\frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} |x\rangle(|f(x)\rangle - |1\oplus f(x)\rangle)$$

Ahora podemos aprovecharnos de una bonita propiedad matemática. Para cualquier input $x$, el valor de $f(x)$ es o $0$ o $1$. Si es $0$, el estado del último qubit es el mismo que antes de pasar por el bloque $U_f$. Y si es $1$, el estado es $\frac{1}{\sqrt{2}}(-|0\rangle+|1\rangle)$, es decir, es el mismo, pero con un signo negativo. Consecuentemente, podemos escribir el output tras $U_f$ como

$$|\psi\rangle=\frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} |x\rangle(|f(x)\rangle - |1\oplus f(x)\rangle)=\frac{1}{\sqrt{2^{n+1}}}\sum_{x=0}^{2^n-1} (-1)^{f(x)}|x\rangle(|0\rangle - |1\rangle)$$

Ahora aplicamos puertas de Hadamard de nuevo a todos los qubits excepto el último, del cual nos podemos ya olvidar. Tras unos simples cálculos, se puede ver que el estado final del sistema es

$$|\psi\rangle=\frac{1}{2^{n}}\sum_{y=0}^{2^n-1}\left[\sum_{x=0}^{2^n-1} (-1)^{f(x)+x · y}\right]|y\rangle,$$

donde $x · y=x_0 y_0 \oplus x_1 y_1 \oplus\dots\oplus x_n y_n$.

Por último, medimos todos los qubits (excepto el último, del cual ya nos habíamos olvidado), y miramos si obtenemos la cadena $y=00\dots 0$. La probabilidad de obtener este resultado es

$$p(00\dots 0)=\left|\frac{1}{2^n}\sum_{x=0}^{2^n-1}(-1)^{f(x)}\right|^2$$

Si la función $f$ es constante, entonces $f(x)=0$ o $f(x)=1$ para todos los $x$, de tal manera que $p(00\dots 0)=1$. Por otro lado, si $f$ es balanceada, habrá tantos términos positivos como negativos en el sumatorio, de tal manera que la suma dará $p(00\dots 0)=0$.

Así que la interpretación es la siguiente: si tras las mediciones se obtiene la cadena $y=00\dots 0$, la función $f$ es constante, y si no, $f$ es balanceada. Y esto, recordemos, ejecutando una sola vez el algoritmo, utilizando únicamente una vez la función $f$ (en el bloque $U_f$), y utilizando la superposición cuántica.

### Programación utilizando IBM Qiskit
Vamos ahora a la parte divertida. Veamos un ejemplo de uso del algoritmo de Deutsch-Jozsa para una función específica $f$ sobre cadenas de bits de longitud $3$. La función será $f(x)=x_0\oplus x_1x_2$. Para saber si esta función es balanceada o no, en un ordenador clásico necesitaríamos evaluar la función hasta $5$ veces, pero necesitaremos solo una si utilizamos un ordenador cuántico.

El primer paso es crear el circuito cuántico en el que escribiremos el programa.

In [ ]:
n          = 3
dj_circuit = QuantumCircuit(n+1, n, name='Deutsch-Jozsa')

En los circuitos cuánticos los qubits siempre comienzan en el estado $|0\rangle$ (por convención). El primer paso es aplicar la puerta $X$ (el equivalente a la clásica NOT) en el último qubit para ponerlo en el estado $|1\rangle$. Tras esto, podemos aplicar las puertas de Hadamard.

In [ ]:
# Aplicar X al último qubit
dj_circuit.x(n)
dj_circuit.barrier()    # Barrera, para mejorar visualización, sin otro objetivo

# Aplicar Hadamard a cada qubit
for qubit in range(n):
    dj_circuit.h(qubit)

Ahora toca aplicar la función mediante el bloque $U_f$. En teoría, esto sería una caja negra, dada por otra persona, que ejecutaría la operación $|x\rangle|y\rangle\rightarrow|x\rangle|y\oplus f(x)\rangle$. En este ejemplo, la tendremos que crear nosotros. Queremos implementar la función $f(x)=x_0\oplus x_1x_2$, cuya tabla de valores es

|$x_0$|$x_1$|$x_2$|$x_0\oplus x_1x_2$|
|-----|-----|-----|:-----------------:|
| $0$ | $0$ | $0$ |        $0$        |
| $0$ | $0$ | $1$ |        $0$        |
| $0$ | $1$ | $0$ |        $0$        |
| $0$ | $1$ | $1$ |        $1$        |
| $1$ | $0$ | $0$ |        $1$        |
| $1$ | $0$ | $1$ |        $1$        |
| $1$ | $1$ | $0$ |        $1$        |
| $1$ | $1$ | $1$ |        $0$        |

Es fácil de ver que es una función balanceada, porque da el resultado $0$ para la mitad de los inputs y $1$ para la otra mitad. Para implementarla, utilizaremos la puerta CNOT.

Fijándonos un poco, es posible ver que esto es simplemente la operación $CNOT(|x\rangle|y\rangle)=|x\rangle|y\oplus x\rangle$. Consecuentemente, si hacemos una CNOT entre el primer qubit y el último, podemos crear la parte que calcula $\oplus x_0$

In [ ]:
dj_circuit.barrier()    # De nuevo, para que la visualizacin despus se vea bien

# CNOT entre x0 e y
dj_circuit.cx(0, n)

Lo siguiente es hacer la parte $\oplus x_1x_2$. Para esto, sería muy deseable tener una "puerta $X$ con doble control", que solamente ejecutara la puerta $X$ en el objetivo si los dos qubits de control estuvieran en el estado $|1\rangle$ (esto es, cuando $x_1x_2=1$). Resulta que esta puerta existe, y se llama [puerta de Toffoli](https://en.wikipedia.org/wiki/Toffoli_gate). Podemos aplicarla con los qubits $|x_1\rangle$ y $|x_2\rangle$ como controles, y usando el qubit $|y\rangle$ como objetivo.

In [ ]:
# Toffoli entre x1, x2 e y
dj_circuit.ccx(1, 2, n)

dj_circuit.barrier()

La siguiente parte del protocolo es aplicar la puerta de Hadamard en todos los qubits (excepto el último, del cual nos podemos olvidar)

In [ ]:
# Aplicar Hadamard a todos los qubits excepto el último
for qubit in range(n):
    dj_circuit.h(qubit)
dj_circuit.barrier()

Y como paso final, hacemos medidas sobre el estado de los qubits. Para especificar una medición, tenemos que decir qué qubit queremos medir, y dónde (en qué registro clásico) almacenaremos el resultado.

In [ ]:
for i in range(n):
    dj_circuit.measure(i, i)

¡Y ya está! Ahora podemos, por ejemplo, observar la apariencia del circuito que hemos construido. Los estados cuánticos aparecen representados con líneas simples, mientras que la información clásica se representa con líneas dobles. Las barreras grises son las ```barrier```s que hemos estado definiendo a lo largo del circuito. Son elementos que permiten tener el circuito bien ordenado y separado en partes, pero no tienen mayor significancia.

In [ ]:
dj_circuit.draw(output='mpl')

## Ejecución del algoritmo
#### En simuladores clásicos
**Ejercicio**: Ejecuta una vez (modificando el parámetro ``shots``) el circuito en el ``qasm_simulator``, y determina si la función $f$ es balanceada o no.

Hemos obtenido el resultado con una sola ejecución del programa. Para comparar, si hubiéramos comenzado a probar inputs en un ordenador clásico, habríamos necesitado al menos dos ejecuciones y, en el peor de los casos, cinco.

#### En ordenadores cuánticos
Una buena característica de ```qiskit``` es que acceder a ordenadores cuánticos reales es tan sencillo como utilizar los simuladores clásicos. Hay un pequeño paso extra, que es 'iniciar sesión' en IBM Q Experience. Esto se lleva a cabo a través del módulo ``IBMQ``, y requiere la token asociada a [tu cuenta](https://quantumexperience.ng.bluemix.net/qx/account/advanced).

In [ ]:
QX_TOKEN = 'YOUR_TOKEN'
IBMQ.enable_account(QX_TOKEN)

Ahora podemos ver qué ordenadores están disponibles para ejecutar programas.

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
print("%20s" % "Name", "|", "N. qubits")
print("-------------------------------")
for backend in provider.backends():
  print("%20s" % backend.name(), "|", backend.configuration().n_qubits)

Una vez hayamos elegido uno, los pasos para ejecutar un programa son los mismos que cuando utilizamos simuladores.

In [ ]:
real_chip = provider.get_backend('ibmq_ourense')
result = execute(dj_circuit, backend=real_chip, shots=1024)
print(result.result().get_counts())

Te habrás dado cuenta de que ahora estamos haciendo no una sola ejecución del algoritmo, sino $1024$. Esto es para mostrar que, en algunos casos, podemos obtener el resultado $000$ cuando tenemos una función balanceada. Esto ocurre porque los ordenadores cuánticos actuales son aparatos muy sensibles, e incluso una fuente pequeña de ruido afecta el resultado de los cálculos. A pesar de que este es uno de los grandes problemas a los que se tiene que enfrentar la computación cuántica, existen ya soluciones teóricas universales, así como métodos experimentales para solventarlo.

**Ejercicio:** Implementa nuevas funciones, balanceadas y constantes, y comprueba mediante el algoritmo de Deutsch-Jozsa de qué tipo son.

*Sugerencias*: $f(x)= x_0\oplus x_1\oplus x_2$, $f(x)= x_0x_1\oplus x_1x_2\oplus x_0x_2$, $f(x)= 0$, $f(x)= 1$

## Conclusiones y más allá
En este taller hemos aprendido los instrumentos básicos en computación cuántica y con ellos hemos aprendido las propiedades fundamentales de la física cuántica: la superposición de estados, el colapso de la función de onda, y el entrelazamiento. Además, hemos visto un ejemplo de cómo estas propiedades se pueden explotar para realizar cálculos con muchísimas menos operaciones que utilizando ordenadores clásicos.

Esto es solo el comienzo. La rama de computación cuántica está viviendo un momento de gran auge, motivado en gran parte por la entrada de grandes empresas tecnológicas y por la aparición de los primeros prototipos de ordenadores cuánticos accesibles para el gran público. A continuación recopilo una lista no exhaustiva de lecturas y recursos de interés:
  - Libros de texto, cursos y lecturas
    - [Quantum Computation and Quantum Information](ww.cambridge.org/9781107002173), de Michael Nielsen e Isaac Chuang. Poco más que decir que ha sido el libro de texto de referencia desde hace casi ya 20 años.
    - [Learn quantum computation using Qiskit](https://community.qiskit.org/textbook/). Un libro de texto desarrollado por IBM para introducir a la computación cuántica a través de su librería de programación.
    - El curso de [Quantum Machine Learning](https://courses.edx.org/courses/course-v1:University_of_TorontoX+UTQML101x+1T2019/course/) de la Universidad de Toronto. A pesar de que el curso no está activo, todo el contenido se puede encontrar en [el GitLab oficial](https://gitlab.com/qosf/qml-mooc).
    - [Quantum Computing in the NISQ era and beyond](https://quantum-journal.org/papers/q-2018-08-06-79/), de John Preskill. Un interesante artículo, de relativamente fácil lectura, acerca de los problemas a los que el campo de computación cuántica se tendrá que enfrentar en el futuro próximo.
  - Plataformas de computación abiertas
    - [IBM Q Experience](https://quantum-computing.ibm.com/). La plataforma de acceso a los ordenadores cuánticos de IBM.
    - [D-Wave Leap](https://cloud.dwavesys.com/leap/). La plataforma de acceso a las máquinas de temple cuántico de D-Wave. Estas máquinas realizan cálculos en otro modelo de computación, diferente al modelo de circuitos, lo cual tiene sus ventajas e inconvenientes.
    - [Azure Quantum](https://azure.microsoft.com/services/quantum). La plataforma de Microsoft para acceder a los ordenadores cuánticos de [IonQ](https://ionq.com/), [Honeywell](https://www.honeywell.com/en-us/company/quantum), y [Quantum Circuits Inc.](https://quantumcircuits.com/)
  - Librerías de programación para plataformas específicas
    - [Qiskit](www.qiskit.org), para los ordenadores de IBM. Es la que hemos utilizado en este taller, y de hecho es una de las más completas, con una gran cantidad de funcionalidades y de [algoritmos preconstruidos](https://qiskit.org/aqua).
    - [Rigetti Forest](https://www.rigetti.com/forest), para los ordenadores de Rigetti, a los cuales se puede [acceder por invitación](https://www.rigetti.com/qcs).
    - [D-Wave Ocean](https://ocean.dwavesys.com/), para simular y trabajar con las máquinas de D-Wave.
    - [StrawberryFields](https://www.xanadu.ai/software/), de Xanadu AI. Permite simular (y en el futuro, ejecutar en chips reales) ordenadores cuánticos basados en sistemas continuos en lugar de qubits.
    - [Cirq](https://github.com/quantumlib/Cirq), de Google. De momento solo sirve para realizar simulaciones, pero se espera que sera la manera de hacer interfaz con sus procesadores cuando éstos se hagan disponibles al público.
  - Librerías de programación genéricas
    - [Q#](https://www.microsoft.com/en-us/quantum/development-kit). La librería de Microsoft, lo más parecido a C# que se puede ser. Viene acompañada por [un repositorio con tutoriales](https://github.com/Microsoft/QuantumKatas) y programas para problemas comunes.
    - [PennyLane](https://www.xanadu.ai/software/), de Xanadu AI. Es una librería dedicada a quantum machine learning, permitiendo interfaces entre librerías de machine learning clásico como Tensorflow o Pytorch y ordenadores cuánticos de una manera agnóstica, de tal manera que no está restringido a ninguna empresa o arquitectura en particular.